In [228]:
import pandas as pd
import pickle, bz2
import wget
import itertools, os, shutil, glob

from multiprocessing.pool import ThreadPool
pd.options.display.max_colwidth = 1000

## Load pkl file from CHOCOPhlAn database, read taxonomy and get GenBank (GCA) ids.
Something is wrong with `t__Microcystis_aeruginosa_Micro_LB_01|t__PRJNA18127`

In [218]:
# Load pkl file form CHOCOPhlAn database
mpa_pkl_loc = '/data11/bio/databases/metaphlan2/mpa_v30_CHOCOPhlAn_201901/mpa_v30_CHOCOPhlAn_201901.pkl'

with open(mpa_pkl_loc, 'rb' ) as a:
    mpa_pkl = pickle.loads( bz2.decompress( a.read() ) )
    
mpa_pkl.keys()

dict_keys(['markers', 'taxonomy', 'merged_taxon'])

In [219]:
wrong = [
    'k__Bacteria|p__Proteobacteria|c__Alphaproteobacteria|o__Rickettsiales|f__Rickettsiaceae|g__Rickettsia|s__Rickettsia_parkeri|t__GCA_000007025',
    'k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Thiotrichales|f__Francisellaceae|g__Francisella|s__Francisella_hispaniensis|t__GCA_000008985',
    'k__Bacteria|p__Proteobacteria|c__Alphaproteobacteria|o__Rickettsiales|f__Anaplasmataceae|g__Ehrlichia|s__Ehrlichia_muris|t__GCA_000013145',
    'k__Bacteria|p__Proteobacteria|c__Alphaproteobacteria|o__Rickettsiales|f__Rickettsiaceae|g__Rickettsia|s__Rickettsia_rhipicephali|t__GCA_000014345',
    'k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Thiotrichales|f__Francisellaceae|g__Francisella|s__Francisella_hispaniensis|t__GCA_000014645',
    'k__Bacteria|p__Proteobacteria|c__Alphaproteobacteria|o__Rickettsiales|f__Rickettsiaceae|g__Rickettsia|s__Rickettsia_rhipicephali|t__GCA_000016625',
    'k__Bacteria|p__Proteobacteria|c__Alphaproteobacteria|o__Rickettsiales|f__Rickettsiaceae|g__Rickettsia|s__Rickettsia_amblyommatis|t__GCA_000016625',
    'k__Bacteria|p__Proteobacteria|c__Alphaproteobacteria|o__Rickettsiales|f__Rickettsiaceae|g__Rickettsia|s__Rickettsia_parkeri|t__GCA_000017445',
    'k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Thiotrichales|f__Francisellaceae|g__Francisella|s__Francisella_noatunensis|t__GCA_000019285',
    'k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Thiotrichales|f__Francisellaceae|g__Francisella|s__Francisella_hispaniensis|t__GCA_000019285',
    'k__Bacteria|p__Proteobacteria|c__Alphaproteobacteria|o__Rickettsiales|f__Rickettsiaceae|g__Rickettsia|s__Rickettsia_parkeri|t__GCA_000021525',
    'k__Bacteria|p__Proteobacteria|c__Alphaproteobacteria|o__Rickettsiales|f__Rickettsiaceae|g__Rickettsia|s__Rickettsia_parkeri|t__GCA_000023005',
    'k__Bacteria|p__Proteobacteria|c__Alphaproteobacteria|o__Rickettsiales|f__Rickettsiaceae|g__Rickettsia|s__Rickettsia_amblyommatis|t__GCA_000023005',
    'k__Bacteria|p__Proteobacteria|c__Alphaproteobacteria|o__Rickettsiales|f__Rickettsiaceae|g__Rickettsia|s__Rickettsia_parkeri|t__GCA_000166935',
    'k__Bacteria|p__Proteobacteria|c__Alphaproteobacteria|o__Rickettsiales|f__Rickettsiaceae|g__Rickettsia|s__Rickettsia_raoultii|t__GCA_000166935',
    'k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Thiotrichales|f__Francisellaceae|g__Francisella|s__Francisella_hispaniensis|t__GCA_000195535',
    'k__Bacteria|p__Planctomycetes|c__Planctomycetia|o__Planctomycetales|f__Planctomycetaceae|g__Rhodopirellula|s__Rhodopirellula_europaea|t__GCA_000196115',
    'k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Thiotrichales|f__Francisellaceae|g__Francisella|s__Francisella_noatunensis|t__GCA_000219045', #?
    'k__Bacteria|p__Proteobacteria|c__Alphaproteobacteria|o__Rickettsiales|f__Rickettsiaceae|g__Rickettsia|s__Rickettsia_raoultii|t__GCA_000237845',
    'k__Bacteria|p__Planctomycetes|c__Planctomycetia|o__Planctomycetales|f__Planctomycetaceae|g__Rhodopirellula|s__Rhodopirellula_europaea|t__GCA_001027925',
    'k__Bacteria|p__Actinobacteria|c__Coriobacteriia|o__Coriobacteriales|f__Atopobiaceae|g__Atopobium|s__Atopobium_vaginae|t__GCA_001398375',
    'k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactobacillales|f__Lactobacillaceae|g__Lactobacillus|s__Lactobacillus_rogosae|t__GCA_001405835',
    'k__Bacteria|p__Actinobacteria|c__Actinobacteria|o__Micrococcales|f__Intrasporangiaceae|g__Janibacter|s__Janibacter_terrae|t__GCA_001889125',
    'k__Bacteria|p__Tenericutes|c__Mollicutes|o__Mycoplasmatales|f__Mycoplasmataceae|g__Mycoplasma|s__Mycoplasma_wenyonii|t__GCA_002705755',
    'k__Bacteria|p__Planctomycetes|c__Planctomycetia|o__Planctomycetales|f__Planctomycetaceae|g__Gimesia|s__Gimesia_maris|t__GCA_002706765',
    'k__Bacteria|p__Spirochaetes|c__Spirochaetia|o__Spirochaetia_unclassified|f__Leptospiraceae|g__Leptospira|s__Leptospira_hartskeerlii|t__GCA_002811475',
]

In [220]:
# Get taxons
taxons = list(set(mpa_pkl['taxonomy'].keys()))

organisms = [{"full_tax_name": t, 'GCA_acc': t.split('|t__')[-1]} for t in taxons]
organisms = pd.DataFrame(organisms).set_index('GCA_acc')

# Select only bacteria and archaea
organisms = organisms.loc[(organisms.full_tax_name.str.contains('k__Archaea') | organisms.full_tax_name.str.contains('k__Bacteria'))]

organisms = organisms.loc[~organisms['full_tax_name'].isin(wrong)]
organisms['s__'] = organisms['full_tax_name'].str.split('|').str[-2]
gca_accs_set = set(organisms.index)

organisms.shape

(25458, 2)

In [77]:
# markers = list(mpa_pkl['markers'].keys())
# markers[0]
# mpa_pkl['markers']['244590__GeneID:2658371']

{'ext': [],
 'score': 0.0,
 'clade': 's__Sulfolobus_spindle_shaped_virus_2',
 'len': 147,
 'taxon': 'k__Viruses|p__Viruses_unclassified|c__Viruses_unclassified|o__Viruses_unclassified|f__Fuselloviridae|g__Alphafusellovirus|s__Sulfolobus_spindle_shaped_virus_2'}

In [93]:
# clades = []

# for k, v in mpa_pkl['markers'].items():
#     clades.append(v['clade'])
    
# clades = list(set(clades))
# mpa_pkl['markers']['244590__GeneID:2658371']

{'ext': [],
 'score': 0.0,
 'clade': 's__Sulfolobus_spindle_shaped_virus_2',
 'len': 147,
 'taxon': 'k__Viruses|p__Viruses_unclassified|c__Viruses_unclassified|o__Viruses_unclassified|f__Fuselloviridae|g__Alphafusellovirus|s__Sulfolobus_spindle_shaped_virus_2'}

In [94]:
taxa = 'k__Bacteria|p__Tenericutes|c__Mollicutes|o__Mycoplasmatales|f__Mycoplasmataceae|g__Mycoplasma|s__Mycoplasma_wenyonii|t__GCA_002705755'
print(mpa_pkl['taxonomy'][taxa])
clade = 's__Mycoplasma_wenyonii'
# for k, v in mpa_pkl['markers'].items():
#     if v['clade'] == clade:
#         print(k, v)

('2|544448|31969|2085|2092|2093|65123', 3625508)


In [84]:
taxa = 'k__Bacteria|p__Actinobacteria|c__Actinobacteria|o__Micrococcales|f__Microbacteriaceae|g__Microbacterium|s__Microbacterium_esteraromaticum|t__GCA_002705755'
print(mpa_pkl['taxonomy'][taxa])

taxa = 'k__Bacteria|p__Actinobacteria|c__Actinobacteria|o__Micrococcales|f__Microbacteriaceae|g__Microbacterium|s__Microbacterium_esteraromaticum'
clade = 's__Microbacterium_esteraromaticum'
for k, v in mpa_pkl['markers'].items():
    if 'Microbacterium_esteraromaticum' in v['taxon']:
        print(k, v)

('2|201174|1760|85006|85023|33882|57043', 3625508)


# PROBLEMS WITH DUPLICATED GCA_ACCs!!

In [221]:
organisms['dup'] = organisms.index.duplicated(keep=False)
organisms.loc[organisms['dup'] == True].sort_index()

,full_tax_name,s__,dup
GCA_acc,,,


## Download bacteria and archaea assembly summary from GenBank FTP

In [50]:
!wget ftp://ftp.ncbi.nih.gov/genomes/genbank/bacteria/assembly_summary.txt -O ./bacteria_assembly_summary.txt
!wget ftp://ftp.ncbi.nih.gov/genomes/genbank/archaea/assembly_summary.txt -O ./archaea_assembly_summary.txt

--2020-06-14 19:46:44--  ftp://ftp.ncbi.nih.gov/genomes/genbank/bacteria/assembly_summary.txt
           => ‘./bacteria_assembly_summary.txt’
Resolving ftp.ncbi.nih.gov (ftp.ncbi.nih.gov)... 130.14.250.7, 2607:f220:41e:250::13
Connecting to ftp.ncbi.nih.gov (ftp.ncbi.nih.gov)|130.14.250.7|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /genomes/genbank/bacteria ... done.
==> SIZE assembly_summary.txt ... 208538454
==> PASV ... done.    ==> RETR assembly_summary.txt ... done.
Length: 208538454 (199M) (unauthoritative)

assembly_summary.tx 100%[===================>] 198.88M  1017KB/s    in 83s     

2020-06-14 19:48:09 (2.40 MB/s) - ‘./bacteria_assembly_summary.txt’ saved [208538454]

--2020-06-14 19:48:09--  ftp://ftp.ncbi.nih.gov/genomes/genbank/archaea/assembly_summary.txt
           => ‘./archaea_assembly_summary.txt’
Resolving ftp.ncbi.nih.gov (ftp.ncbi.nih.gov)... 130.14.250.13, 2607:f220:41e:250::7


In [217]:
# Load bacteria summary table
bacteria_assembly_summary = pd.read_csv('./bacteria_assembly_summary.txt', sep = '\t', low_memory=False, header=1, index_col=0)
bacteria_assembly_summary.index = bacteria_assembly_summary.index.str.split('.').str[0]

# Load archaeal summary table
archaea_assembly_summary = pd.read_csv('./archaea_assembly_summary.txt', sep = '\t', low_memory=False, header=1, index_col=0)
archaea_assembly_summary.index = archaea_assembly_summary.index.str.split('.').str[0]

In [222]:
bacteria_archea_summary = pd.concat([bacteria_assembly_summary, archaea_assembly_summary])
accs_in_genbank = set(bacteria_archea_summary.index)
# Get intersection of GCA accesions in mpa_pkl and genbank aseembly summary files
gca_accs_intersection = gca_accs_set.intersection(accs_in_genbank)
# Get accessions that are notpresent in GenBank
no_accs_in_genbank = gca_accs_set - gca_accs_intersection

In [223]:
assemblies_for_download = bacteria_archea_summary.loc[gca_accs_intersection]
# assemblies_for_download = assemblies_for_download.loc[~assemblies_for_download.ftp_path.isna()]
organisms_for_download = organisms.loc[gca_accs_intersection]

assemblies_for_download['ACC'] = assemblies_for_download.index
assemblies_for_download_dict = assemblies_for_download.to_dict(orient='records')

In [224]:
assemblies_for_download.shape

(25397, 22)

In [292]:
dwn_dir = '/data11/bio/databases/metaphlan2/mp_genomes'
# https://markhneedham.com/blog/2018/07/15/python-parallel-download-files-requests/

def fetch_url(d):
    acc = d['ACC']
    ftp_path = d['ftp_path'] + '/' + d['ftp_path'].split('/')[-1] + '_genomic.fna.gz'
    
    save_dir = os.path.join(dwn_dir, acc)
    os.makedirs(save_dir, exist_ok=True)
    
    try:
        file_loc = os.path.join(save_dir,  d['ftp_path'].split('/')[-1] + '_genomic.fna.gz')
        wget.download(ftp_path, out=file_loc)
        print(file_loc, os.path.isfile(file_loc))
        return {'acc': acc, 'status': 'ok'}
    except:
        print(acc, 'fail')
        return {'acc': acc, 'status': 'error'}    

In [ ]:
results = ThreadPool(35).imap_unordered(fetch_url, assemblies_for_download_dict)

for path in results:
    if path['status'] != 'ok':
        print(path)

## Select good genomes (one per species)

In [317]:
merged = assemblies_for_download.merge(organisms_for_download, left_index=True, right_index=True,)
len(merged['s__'].unique())
# merged.shape

10360

In [183]:
gcas_with_dups = [
 'GCA_000007025',
 'GCA_000008985',
 'GCA_000013145',
 'GCA_000014345',
 'GCA_000014645',
 'GCA_000016625',
 'GCA_000017445',
 'GCA_000019285',
 'GCA_000021525',
 'GCA_000023005',
 'GCA_000166935',
 'GCA_000195535',
 'GCA_000196115',
 'GCA_000219045',
 'GCA_000237845',
 'GCA_001027925',
 'GCA_001398375',
 'GCA_001405835',
 'GCA_001889125',
 'GCA_002705755',
 'GCA_002706765',
 'GCA_002811475'
]

In [321]:
merged.loc[~merged.index.isin(gcas_with_dups)].shape

(25375, 25)

In [322]:
# First let's get genomes that are representative or reference
reference_genomes = merged.loc[merged['refseq_category'] == 'reference genome']
representative_genomes = merged.loc[merged['refseq_category'] == 'representative genome']

good_genomes = pd.concat([reference_genomes, representative_genomes])

# checking species duplicates
good_genomes['dup'] = good_genomes['s__'].duplicated(keep=False)
good_genomes.loc[good_genomes['dup'] == True].sort_values('s__')[['taxid','species_taxid','full_tax_name', 'organism_name', 's__', 'refseq_category','assembly_level', 'infraspecific_name','isolate','version_status',  'release_type', 'genome_rep']]

,taxid,species_taxid,full_tax_name,organism_name,s__,refseq_category,assembly_level,infraspecific_name,isolate,version_status,release_type,genome_rep
GCA_000943515,31958,31958,k__Bacteria|p__Actinobacteria|c__Actinobacteria|o__Pseudonocardiales|f__Pseudonocardiaceae|g__Amycolatopsis|s__Amycolatopsis_orientalis|t__GCA_000943515,Amycolatopsis orientalis,s__Amycolatopsis_orientalis,representative genome,Complete Genome,strain=B-37,NaN,latest,Major,Full
GCA_000400635,129921,129921,k__Bacteria|p__Actinobacteria|c__Actinobacteria|o__Pseudonocardiales|f__Pseudonocardiaceae|g__Amycolatopsis|s__Amycolatopsis_orientalis|t__GCA_000400635,Amycolatopsis keratiniphila,s__Amycolatopsis_orientalis,representative genome,Complete Genome,strain=HCCB10007,NaN,latest,Major,Full
GCA_000008725,272561,813,k__Bacteria|p__Chlamydiae|c__Chlamydiia|o__Chlamydiales|f__Chlamydiaceae|g__Chlamydia|s__Chlamydia_trachomatis|t__GCA_000008725,Chlamydia trachomatis D/UW-3/CX,s__Chlamydia_trachomatis,reference genome,Complete Genome,strain=D/UW-3/CX,NaN,latest,Major,Full
GCA_000012125,315277,813,k__Bacteria|p__Chlamydiae|c__Chlamydiia|o__Chlamydiales|f__Chlamydiaceae|g__Chlamydia|s__Chlamydia_trachomatis|t__GCA_000012125,Chlamydia trachomatis A/HAR-13,s__Chlamydia_trachomatis,representative genome,Complete Genome,strain=A/HAR-13,NaN,latest,Major,Full
GCA_000093145,363952,363952,k__Bacteria|p__Proteobacteria|c__Betaproteobacteria|o__Burkholderiales|f__Comamonadaceae|g__Comamonas|s__Comamonas_testosteroni|t__GCA_000093145,Comamonas thiooxydans,s__Comamonas_testosteroni,representative genome,Complete Genome,strain=CNB-1 substr. CNB-2,NaN,latest,Major,Full
GCA_000739375,1392005,285,k__Bacteria|p__Proteobacteria|c__Betaproteobacteria|o__Burkholderiales|f__Comamonadaceae|g__Comamonas|s__Comamonas_testosteroni|t__GCA_000739375,Comamonas testosteroni TK102,s__Comamonas_testosteroni,representative genome,Complete Genome,strain=TK102,NaN,latest,Major,Full
GCA_000017105,434922,777,k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Legionellales|f__Coxiellaceae|g__Coxiella|s__Coxiella_burnetii|t__GCA_000017105,Coxiella burnetii Dugway 5J108-111,s__Coxiella_burnetii,representative genome,Complete Genome,strain=Dugway 5J108-111,NaN,latest,Major,Full
GCA_000007765,227377,777,k__Bacteria|p__Proteobacteria|c__Gammaproteobacteria|o__Legionellales|f__Coxiellaceae|g__Coxiella|s__Coxiella_burnetii|t__GCA_000007765,Coxiella burnetii RSA 493,s__Coxiella_burnetii,reference genome,Complete Genome,strain=RSA 493,NaN,latest,Major,Full
GCA_900142135,1121393,259354,k__Bacteria|p__Proteobacteria|c__Deltaproteobacteria|o__Desulfobacterales|f__Desulfobacteraceae|g__Desulfatibacillum|s__Desulfatibacillum_alkenivorans|t__GCA_900142135,Desulfatibacillum alkenivorans DSM 16219,s__Desulfatibacillum_alkenivorans,representative genome,Scaffold,strain=DSM 16219,NaN,latest,Major,Full
GCA_000021905,218208,218208,k__Bacteria|p__Proteobacteria|c__Deltaproteobacteria|o__Desulfobacterales|f__Desulfobacteraceae|g__Desulfatibacillum|s__Desulfatibacillum_alkenivorans|t__GCA_000021905,Desulfatibacillum aliphaticivorans,s__Desulfatibacillum_alkenivorans,representative genome,Complete Genome,strain=AK-01,NaN,latest,Major,Full


In [323]:
remove_from_good = [
    'GCA_000400635',
    'GCA_000012125',
    'GCA_000093145',
    'GCA_000017105',
    'GCA_000021905',
    'GCA_000152245',
    'GCA_002763695',
    'GCA_900453235',
    'GCA_000934565',
    'GCA_001708485',
    'GCA_900129835',
    'GCA_000517305',
    'GCA_001400735',
    'GCA_002905795',
    'GCA_001664265',
    'GCA_000725365',
]
good_genomes = good_genomes.loc[~good_genomes.index.isin(remove_from_good)]

In [324]:
managed_clades = good_genomes['s__'].unique()
now_manage_this = merged[~merged['s__'].isin(managed_clades)]

In [325]:
now_manage_this['dup'] = now_manage_this['s__'].duplicated(keep=False)

wo_dups = now_manage_this.loc[now_manage_this['dup'] != True]

now_manage_this_dups = now_manage_this.loc[now_manage_this['dup'] == True].sort_values('s__')

/home/fedorov/miniconda3/envs/notebook/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [326]:
now_manage_this_dups['dup'] = now_manage_this_dups['s__'].duplicated(keep='first')
dups_first = now_manage_this_dups.loc[now_manage_this_dups['dup'] == False]

In [327]:
all_managed  = pd.concat([good_genomes, wo_dups, dups_first])

In [311]:
all_managed.shape
len(all_managed['s__'].unique())

10360

In [328]:
all_managed['file_name'] = all_managed['ftp_path'].str.split('/').str[-1] + '_genomic.fna'

# Preparations on FS for Phylophlan
1 - ungz all genomes

In [280]:
!snakemake -s ./ungz_genomes.py -n run_all -j 20

Building DAG of jobs...
Nothing to be done.


2 - move to flat new folder 

In [281]:
!mv /data11/bio/databases/metaphlan2/mp_genomes/*/*.fna /data11/bio/databases/metaphlan2/mp_genomes_fna

mv: cannot stat '/data11/bio/databases/metaphlan2/mp_genomes/*/*.fna': No such file or directory


In [216]:
all_managed.loc[all_managed['file_name'].str.contains('GCA_002077945')]

,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,version_status,...,gbrs_paired_asm,paired_asm_comp,ftp_path,excluded_from_refseq,relation_to_type_material,ACC,dup,full_tax_name,s__,file_name
GCA_002077945,PRJNA316778,SAMN04590022,LVYD00000000.1,representative genome,1703345,1703345,Niastella vici,strain=DJ57,NaN,latest,...,GCF_002077945.1,identical,ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/002/077/945/GCA_002077945.1_ASM207794v1,NaN,assembly from type material,GCA_002077945,False,k__Bacteria|p__Bacteroidetes|c__Chitinophagia|o__Chitinophagales|f__Chitinophagaceae|g__Niastella|s__Niastella_vici|t__GCA_002077945,s__Niastella_vici,GCA_002077945.1_ASM207794v1_genomic.fna


In [329]:
fasta_files =[f.split('/')[-1] for f in glob.glob('/data11/bio/databases/metaphlan2/mp_genomes_fna/*')]

In [330]:
not_downloaded = [f.split('.')[0] for f in list(set(all_managed['file_name']) - set(fasta_files))]
not_downloaded

[]

In [300]:
not_downloaded = all_managed.loc[not_downloaded]

In [307]:
all_managed.shape

(10360, 26)

In [301]:
ff = []
for d in not_downloaded.to_dict(orient='records'):
    print(fetch_url(d))
    acc = d['ACC']
    ftp_path = d['ftp_path'] + '/' + d['ftp_path'].split('/')[-1] + '_genomic.fna.gz'
    
    save_dir = os.path.join(dwn_dir, acc)
    
    file_loc = os.path.join(save_dir,  d['ftp_path'].split('/')[-1] + '_genomic.fna')
    ff.append(file_loc)

/data11/bio/databases/metaphlan2/mp_genomes/GCA_001434655/GCA_001434655.1_ASM143465v1_genomic.fna.gz True
{'acc': 'GCA_001434655', 'status': 'ok'}
/data11/bio/databases/metaphlan2/mp_genomes/GCA_002953935/GCA_002953935.1_ASM295393v1_genomic.fna.gz True
{'acc': 'GCA_002953935', 'status': 'ok'}
/data11/bio/databases/metaphlan2/mp_genomes/GCA_900114345/GCA_900114345.1_IMG-taxon_2609459760_annotated_assembly_genomic.fna.gz True
{'acc': 'GCA_900114345', 'status': 'ok'}
/data11/bio/databases/metaphlan2/mp_genomes/GCA_000317655/GCA_000317655.1_ASM31765v1_genomic.fna.gz True
{'acc': 'GCA_000317655', 'status': 'ok'}
/data11/bio/databases/metaphlan2/mp_genomes/GCA_000239775/GCA_000239775.2_ASM23977v2_genomic.fna.gz True
{'acc': 'GCA_000239775', 'status': 'ok'}
/data11/bio/databases/metaphlan2/mp_genomes/GCA_001189345/GCA_001189345.1_ASM118934v1_genomic.fna.gz True
{'acc': 'GCA_001189345', 'status': 'ok'}
/data11/bio/databases/metaphlan2/mp_genomes/GCA_900099815/GCA_900099815.1_IMG-taxon_26227365

In [302]:
ff

['/data11/bio/databases/metaphlan2/mp_genomes/GCA_001434655/GCA_001434655.1_ASM143465v1_genomic.fna',
 '/data11/bio/databases/metaphlan2/mp_genomes/GCA_002953935/GCA_002953935.1_ASM295393v1_genomic.fna',
 '/data11/bio/databases/metaphlan2/mp_genomes/GCA_900114345/GCA_900114345.1_IMG-taxon_2609459760_annotated_assembly_genomic.fna',
 '/data11/bio/databases/metaphlan2/mp_genomes/GCA_000317655/GCA_000317655.1_ASM31765v1_genomic.fna',
 '/data11/bio/databases/metaphlan2/mp_genomes/GCA_000239775/GCA_000239775.2_ASM23977v2_genomic.fna',
 '/data11/bio/databases/metaphlan2/mp_genomes/GCA_001189345/GCA_001189345.1_ASM118934v1_genomic.fna',
 '/data11/bio/databases/metaphlan2/mp_genomes/GCA_900099815/GCA_900099815.1_IMG-taxon_2622736510_annotated_assembly_genomic.fna']

In [309]:
for f in list(all_managed['file_name']):
    shutil.copy(os.path.join('/data11/bio/databases/metaphlan2/mp_genomes_fna',f), '/data11/bio/databases/metaphlan2/mp_genomes_subset_fna')

In [ ]:
phylophlan -i /data11/bio/databases/metaphlan2/mp_genomes_fna \
    -d phylophlan \
    -f ~/ProjectsGit/chokophlan_tree/02_tol.cfg \
    --diversity high \
    --fast \
    -o /data11/bio/databases/metaphlan2/mp_genomes_phylophlan3/output_tree \
    --nproc 100 \
    --verbose 2>&1 | tee /data11/bio/databases/metaphlan2/mp_genomes_phylophlan3/phylophlan.log